In [1]:
import pandas as pd
import numpy as np
import cirpy
from collections import defaultdict


In [261]:
compound_fp = './database/Compound.csv'


content_fp = './database/Content.csv'
drug_info_comb_fp = './database/Drug_info_combined.csv'
food_fp = './database/Food.csv'

pd.set_option('display.max_columns', None)

In [262]:
compound_df = pd.read_csv(compound_fp)
# for i in compound_df:
#     display(i)
#     break
compound_df.head(2)

/Applications/anaconda3/envs/deepddi2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (3,4,5,6,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,public_id,name,moldb_iupac,state,annotation_quality,description,cas_number,moldb_inchikey,moldb_inchi,moldb_smiles,moldb_mono_mass,kingdom,superklass,klass,subklass
0,4,FDB000004,Cyanidin 3-(6''-acetyl-galactoside),NaN,low,Constituent of the leaves of Nymphaea alba [CC...,350602-26-5,[H][C@]1(COC(C)=O)O[C@@]([H])(OC2=CC3=C(O)C=C(...,InChI=1S/C23H22O12/c1-9(24)32-8-18-19(29)20(30...,491.118951,HBXXDBKJLPLXPR-DLBZZEGUSA-O,"3-{[(2S,3R,4S,5R,6R)-6-[(acetyloxy)methyl]-3,4...",Organic compounds,Phenylpropanoids and polyketides,Flavonoids,Flavonoid glycosides
1,13,FDB000013,Cyanidin 3-(6''-succinyl-glucoside),NaN,low,Constituent of Phragmites australis [CCD]. Cya...,216692-08-9,[H][C@]1(COC(=O)CCC(O)=O)O[C@@]([H])(OC2=CC3=C...,InChI=1S/C25H24O14/c26-11-6-14(28)12-8-17(24(3...,549.124431,MIYGQTFETYBMKF-WVXUANQFSA-O,"7-methoxy-2,2-dimethyl-2H-chromene-6-carboxyli...",Organic compounds,Phenylpropanoids and polyketides,Flavonoids,Flavonoid glycosides


In [264]:
food_compounds = pd.DataFrame([])
content_df = pd.read_csv(content_fp, chunksize = 200_000)

cnt = 0
for chunk in content_df:
#     cnt += 1
    foo = chunk.loc[chunk.source_type == 'Compound', ['source_id',
                                                    'food_id', 
                                                    'orig_food_common_name',
                                                    'orig_content',
                                                    'orig_unit']]
    foo = foo.drop_duplicates().dropna(subset=['orig_food_common_name'])
#     display(foo)
    
    foo['orig_food_common_name'] = foo\
        .orig_food_common_name.apply(lambda x: x.split(',')[0].lower().strip())
    foo = foo\
            .groupby('orig_food_common_name')\
            .apply(lambda df: df.nlargest(24, 'orig_content'))
#     display(foo)
    food_compounds = pd.concat([food_compounds, foo], axis=0)
#     if cnt == 1:
#         break
# compound_df['']


In [265]:
compound_df_proc = compound_df[['id', 'name', 'cas_number']]\
                            .dropna(subset = ['cas_number'])\
                            .drop_duplicates()
combo = food_compounds.merge(compound_df_proc, left_on = 'source_id', right_on = 'id').drop_duplicates()
final_combo = combo.groupby(['orig_food_common_name', 'name'])\
        .apply(lambda df: pd.DataFrame({'orig_content':[np.mean(df.orig_content)],
                                        'orig_unit': df.orig_unit.values[0],
                                        'cas_number': df.cas_number.values[0]}))\
        .reset_index()

In [273]:
final_combo_fp = './database/food_compounds_lookup.csv'
# final_combo.drop(columns = ['level_2']).to_csv(final_combo_fp, index = None)
final_combo = pd.read_csv(final_combo_fp)

In [274]:
final_combo.loc[final_combo.orig_food_common_name == 'orange']#.groupby(['orig_food_common_name']).count().max()

,orig_food_common_name,name,orig_content,orig_unit,cas_number
15159,orange,(E)-2-Phenyl-2-butenal,59.000000,mg/100 g,C\C=C(\C=O)C1=CC=CC=C1
15160,orange,(R)-Oxypeucedanin,22.100000,mg/100 g,CC1(C)OC1COC1=C2C=CC(=O)OC2=CC2=C1C=CO2
15161,orange,(S)-Naringenin,4040.000000,mg/100g,[H][C@]1(CC(=O)C2=C(O1)C=C(O)C=C2O)C1=CC=C(O)C=C1
15162,orange,"(Z,Z)-9,12-Octadecadienoic acid",168.000000,mg/100g,[H]\C(CCCCC)=C(/[H])C\C([H])=C(/[H])CCCCCCCC(O)=O
15163,orange,17alpha-ethynylestradiol,1000.000000,mg/100g,[H][C@@]12CC[C@@](O)(C#C)[C@@]1(C)CC[C@]1([H])...
15164,orange,Arginine,138.966667,mg/100g,N[C@@H](CCCNC(N)=N)C(O)=O
15165,orange,"C18:1, n-9",110.000000,mg/100g,CCCCCCCC\C=C\CCCCCCCC(O)=O
15166,orange,Calcium,163.325000,mg/100g,[Ca++]
15167,orange,D-Aspartic acid,110.000000,mg/100 g,N[C@H](CC(O)=O)C(O)=O
15168,orange,D-Fructose,2260.000000,mg/100g,OCC(O)C(O)C(O)C(=O)CO.OCC1OC(O)(CO)C(O)C1O.OCC...


In [101]:
# compound_df = pd.read_csv(compound_fp)
# for i in compound_df:
#     display(i.head(2).T)
#     tmp = i.loc[i.description == '350602-26-5', 'cas_number'].values
#     display(tmp)
#     break

In [298]:
# drug_info_comb_fp = './data/Drug_info_combined.csv'
# drug_info_comb = pd.read_csv(drug_info_comb_fp, index_col = 'Unnamed: 0')
# compounds = pd.DataFrame(drug_info_comb.Name.str.lower().dropna())
# # compounds.to_csv('./database/compounds.csv', index = None)

food = pd.read_csv('./database/food_compounds_lookup.csv')
pd.DataFrame({'Name': food.orig_food_common_name\
                          .str.replace('(', '').str.replace(')', '')\
                          .str.split()\
                          .str[0].unique()}
            ).to_csv('./database/food.csv', index = None)

food_loc = food.orig_food_common_name\
            .str.replace('(', '')\
            .str.replace(')', '')\
            .str.split().str[0]
comp_loc = food.name.str.lower()

food2comp = defaultdict(set)
comp2food = defaultdict(set)
for f,c in zip(food_loc, comp_loc):
    food2comp[f].add(c)
    comp2food[c].add(f)
# TODO; cache

In [45]:
import re
import pandas as pd
import numpy as np
import os
import subprocess
from collections import defaultdict

# Authors: 
MODEL_DIR = '/Documents/GitHub/deepddi2/'
INPUT_PATH = './DDI_input.txt'
OUTPUT_DIR = './output'
OUTPUT_TXT = 'output/Final_annotated_DDI_result.txt'
SIGNIFICANCE = 0.8
DFI_INPUT_DRUGS = []
DFI_FOOD_list=[]
DDI_OTHER_DRUGS = []

food_comp = pd.read_csv('./database/food_compounds_lookup.csv')
# pd.DataFrame({'Name': food.orig_food_common_name\
#                           .str.replace('(', '').str.replace(')', '')\
#                           .str.split()\
#                           .str[0].unique()}
#             ).to_csv('./database/food.csv', index = None)

food_loc = food_comp.orig_food_common_name\
            .str.replace('(', '')\
            .str.replace(')', '')\
            .str.split().str[0]
comp_loc = food_comp.name.str.lower()

food2comp = defaultdict(set)
comp2food = defaultdict(set)
for f,c in zip(food_loc, comp_loc):
    food2comp[f].add(c)
    comp2food[c].add(f)
    

def regex_search(desc, pools):
    # assume desc is lowercased
    out = []
    for elem in pools:
        pattern = elem.strip().lower()
        if re.search(pattern, desc):
            out.append(elem)
    return out

def ingest_input(input_json, interaction_type, input_fp = INPUT_PATH,
                 compounds_path = './database/drug_info_combined.csv',
                 food_path = './database/food.csv'):
    assert interaction_type.lower() in ['ddi', 'dfi'], 'API not supported'
    first_line = []
    second_line = []
    # handle ddi
    if interaction_type.lower() == 'ddi':
        drug_pools = pd.read_csv(compounds_path).Name.str.lower()
        
        cur_desc = input_json['current_drug']['drug_desc'].lower()
        drug_title = input_json['current_drug']['drug_title']
        
        drug_search = regex_search(cur_desc.lower(), drug_pools)
        assert drug_search, ('Drug: %s not Found' % drug_title)
            
        first_line += [drug_title+'|'+i for i in drug_search]
        for drug in input_json['other_drug']:
            DDI_OTHER_DRUGS.append(drug['drug_title'].lower())
            other_desc = drug['drug_desc'].lower()
            other_search = regex_search(other_desc, drug_pools)
            
            if not other_search:
                print('Drug: %s not Found' % drug['drug_title'])
                continue
            second_line += [drug['drug_title']+'|'+ i for i in other_search]
    # handle dfi
    else:
        drug_pools = pd.read_csv(compounds_path).Name.str.lower()
        food_pools = pd.read_csv(food_path).Name
        for drug in input_json['other_drug']:
            DFI_INPUT_DRUGS.append(drug['drug_title'].lower())
            drug_search = regex_search(drug['drug_desc'].lower(), drug_pools)
#             print(drug['drug_desc'], drug_search)
            if not drug_search:
                print('Drug: %s not Found' % drug['drug_title'])
                continue
            first_line += [drug['drug_title'] + '|' + i for i in drug_search]
        
        for food in input_json['food_list']:
            food_search = food2comp[food.lower()] 
            if not food_search:
                print('Food: %s not Found' % food)
                continue
            DFI_FOOD_list.append(food)
            second_line += [food + '|' + i for i in food_search]
            
    # TODO handle not-found case           
    if os.path.exists(input_fp):
        os.remove(input_fp)
    
    with open(input_fp, 'w') as fw:
        fr = '\t'.join(first_line) + '\n'
        sr = '\t'.join(second_line) + '\n'
        fw.write(fr)
        fw.write(sr)


def run(input_json,interaction_type,thres=SIGNIFICANCE):
    # INPUT: 
    #   input_json: the json file of input info
    #   interactioin_type: 'DFI' or 'DDI'

    # execute & make sure it runs linearly
    ingest_input(input_json, interaction_type)
    cmd = ('python run_DeepDDI.py -i %s -o %s -t %s'%('DDI_input.txt', 'output',str(interaction_type))).split()
    try:
        subprocess.run(cmd)
        return collect_output(thres)
    except AssertionError:
        return None


def collect_output(thres = SIGNIFICANCE, out_txt = OUTPUT_TXT):
    res = pd.read_csv(out_txt,
                      sep='\t', 
                      header=0)[['drug1', 'drug2',
                                          'DDI_prob', 'DDI_prob_std',
                                          'Confidence_DDI', 'Sentence',
                                          'Side effects (left)',
                                          'Side effects (right)']]
    
    temp = res.loc[(res.Confidence_DDI == 1) &\
                   (res.DDI_prob >= thres)]
    
    if DDI_OTHER_DRUGS:
        temp = temp.loc[(res.drug1.str.contains(r'|'.join(DDI_OTHER_DRUGS))) |\
                        (res.drug2.str.contains(r'|'.join(DDI_OTHER_DRUGS)))]
        
    if DFI_INPUT_DRUGS:
        temp = temp.loc[(res.drug1.str.contains(r'|'.join(DFI_INPUT_DRUGS))) |\
                        (res.drug2.str.contains(r'|'.join(DFI_INPUT_DRUGS)))]
    
    out = {}
    display(temp)
    print(DFI_INPUT_DRUGS)
    out['interactions'] = []
    for line in temp.iterrows():
        inner_out = {}
        row = line[1].values
        other_drug = row[1] ###
        #     print(row)
        drug1=re.findall('(.*)\(.*$',row[0])[0]
        drug2=re.findall('(.*)\(.*$',row[1])[0]
        if DFI_INPUT_DRUGS:
            if drug1 not in DFI_INPUT_DRUGS:
                other_drug = drug1
            elif drug2 not in DFI_INPUT_DRUGS:
                other_drug = drug2
            else:
                other_drug = drug1 ### random
        if DDI_OTHER_DRUGS:
            if drug2 in DDI_OTHER_DRUGS:
                other_drug = drug2
            elif drug1 in DDI_OTHER_DRUGS:
                other_drug = drug1
        
        interaction_desc = row[5]
        side_effect = {}
        # side_effect['probability'] = row[2]
        side_effect['side_effect'] = row[6]
        inner_out['other_drug'] = other_drug
        inner_out['interaction_desc'] = interaction_desc
        inner_out['side_effect'] = side_effect
        out['interactions'].append(inner_out)
    return out

# Example of calling DFI API
dfi_sample_input = {'other_drug': [{'drug_title': 'Aspirin', 'drug_desc': 'Acetaminophen'}, {'drug_title': 'Undyne', 'drug_desc': '? Vitamin C '}],
 'food_list': ['lemon','Apple']}

# Example of calling DDI API
ddi_sample_input =  {'current_drug': {'drug_title': 'Good Drug', 'drug_desc': 'Vitamin C'},
 'other_drug': [{'drug_title': 'Drug A', 'drug_desc': ' cool  Vitamin A '},
                {'drug_title': 'Drug B', 'drug_desc': ' very good Aspirin Acetaminophen'}]}

# ALL you need to call is func 'run(input_json,type)'
# # print(run(ddi_sample_input,'DDI'))
# print(run(dfi_sample_input,'DFI'))

,drug1,drug2,DDI_prob,DDI_prob_std,Confidence_DDI,Sentence,Side effects (left),Side effects (right)
1,lemon(l-ascorbic acid),aspirin(acetaminophen),0.813057,0.232130,1,aspirin(acetaminophen) may increase the antico...,NaN,NaN
8,lemon((-)-isopulegol),aspirin(acetaminophen),0.927107,0.087133,1,The risk or severity of adverse effects can be...,NaN,NaN
17,lemon(jaceosidin),aspirin(acetaminophen),0.864854,0.193616,1,aspirin(acetaminophen) may increase the antico...,NaN,NaN
18,aspirin(acetaminophen),lemon(cholesterol),0.863430,0.194897,1,The metabolism of aspirin(acetaminophen) can b...,NaN,NaN
19,lemon(6-methylscutellarein),aspirin(acetaminophen),0.886880,0.223802,1,aspirin(acetaminophen) may increase the antico...,NaN,NaN
20,aspirin(acetaminophen),lemon(acteoside),0.974245,0.029537,1,The therapeutic efficacy of lemon(acteoside) c...,NaN,NaN
21,lemon(acteoside),aspirin(acetaminophen),0.885869,0.198005,1,aspirin(acetaminophen) may increase the hypogl...,NaN,NaN
23,lemon(gentisic acid),aspirin(acetaminophen),0.978566,0.061513,1,aspirin(acetaminophen) may increase the antico...,NaN,NaN
25,lemon((e)-2-phenyl-2-butenal),aspirin(acetaminophen),0.930095,0.097276,1,The risk or severity of adverse effects can be...,NaN,NaN
34,aspirin(acetaminophen),lemon(d-glucose),0.909916,0.112417,1,The therapeutic efficacy of lemon(d-glucose) c...,NaN,NaN


['aspirin', 'undyne']
{'interactions': [{'other_drug': 'lemon', 'interaction_desc': 'aspirin(acetaminophen) may increase the anticoagulant activities of lemon(l-ascorbic acid).', 'side_effect': {'side_effect': nan}}, {'other_drug': 'lemon(', 'interaction_desc': 'The risk or severity of adverse effects can be increased when lemon((-)-isopulegol) is combined with aspirin(acetaminophen).', 'side_effect': {'side_effect': nan}}, {'other_drug': 'lemon', 'interaction_desc': 'aspirin(acetaminophen) may increase the anticoagulant activities of lemon(jaceosidin).', 'side_effect': {'side_effect': nan}}, {'other_drug': 'lemon', 'interaction_desc': 'The metabolism of aspirin(acetaminophen) can be decreased when combined with lemon(cholesterol).', 'side_effect': {'side_effect': nan}}, {'other_drug': 'lemon', 'interaction_desc': 'aspirin(acetaminophen) may increase the anticoagulant activities of lemon(6-methylscutellarein).', 'side_effect': {'side_effect': nan}}, {'other_drug': 'lemon', 'interaction

In [76]:
def collect_food_output(thres = SIGNIFICANCE, out_txt = OUTPUT_TXT):
    res = pd.read_csv(out_txt,
                      sep='\t', 
                      header=0)[['drug1', 'drug2',
                                          'DDI_prob', 'DDI_prob_std',
                                          'Confidence_DDI', 'Sentence',
                                          'Side effects (left)',
                                          'Side effects (right)']]
    
    temp = res.loc[(res.Confidence_DDI == 1) &\
                   (res.DDI_prob >= thres)]
    
    if DDI_OTHER_DRUGS:
        temp = temp.loc[(res.drug1.str.contains(r'|'.join(DDI_OTHER_DRUGS))) |\
                        (res.drug2.str.contains(r'|'.join(DDI_OTHER_DRUGS)))]
        
    if DFI_INPUT_DRUGS:
        temp = temp.loc[(res.drug1.str.contains(r'|'.join(DFI_INPUT_DRUGS))) |\
                        (res.drug2.str.contains(r'|'.join(DFI_INPUT_DRUGS)))]
    
    out = []
    temp['drug_name']=temp['drug1'].apply(lambda x: re.findall('^[^\(]+',x)[0])
    for i in DFI_FOOD_LIST:
        each_food={}
        drug_interactions=[]
        food_i=temp.loc[temp['drug2'].str.contains(i.lower())]
        find_most=food_i[food_i.groupby('drug_name')['DDI_prob'].transform(max) == food_i['DDI_prob']]
        for row in find_most.iterrows():
            row=row[1]
            each_row={}
            each_row['other_drug_name']=row['drug_name']
            each_row['interaction_desc']=row['Sentence']
            each_row['probability'] = str(float(row['DDI_prob'])*100)[:4]+'%'
            drug_interactions.append(each_row)
        each_food['food_name']=i
        each_food['drug_interactions']=drug_interactions
        out.append(each_food)
    return out
collect_food_output()

[{'drug_interactions': [{'interaction_desc': 'The therapeutic efficacy of lemon(acteoside) can be decreased when used in combination with aspirin(acetaminophen).',
    'other_drug_name': 'aspirin',
    'probability': '97.4%'},
   {'interaction_desc': 'lemon(caffeic acid) may increase the anticoagulant activities of undyne(vitamin c).',
    'other_drug_name': 'undyne',
    'probability': '91.1%'}],
  'food_name': 'Lemon'},
 {'drug_interactions': [{'interaction_desc': 'The therapeutic efficacy of apple(d-fructose) can be decreased when used in combination with aspirin(acetaminophen).',
    'other_drug_name': 'aspirin',
    'probability': '88.6%'}],
  'food_name': 'Apple'}]

In [27]:
dfi_sample_input = {'current_drug': {'drug_title': 'Bad Drug', 'drug_desc': '? Acetaminophen  '},
 'food': ['lemon', 'orange']}

ddi_sample_input =  {'current_drug': {'drug_title': 'Good Drug', 'drug_desc': '! Biotin !'},
 'other_drug': [{'drug_title': 'So-so Drug', 'drug_desc': ' cool  Vitamin C '},
                {'drug_title': 'Pain-Killer', 'drug_desc': ' very good Aspirin Acetaminophen'}]}


In [28]:
print(run(dfi_sample_input,'DFI'))

None


In [ ]:
drug1=re.findall('(.*)\(.*$',row[0])[0]
        drug2=re.findall('(.*)\(.*$',row[1])[0]